In [1]:
# reasonings:
# 1. CNN is an overkill for this task. 
# 2. rule-based algo is more suitable for this task, it has the advantage of knowing the pattern
# 3. presidio also uses rule-based pattern matching: https://github.com/microsoft/presidio/tree/main/presidio-analyzer/presidio_analyzer/predefined_recognizers


# 1. Zip Code Recognizer

In [2]:
# pip install zipcodes
import zipcodes
import pandas as pd
import random
import re

# the train data has 42633 records
train_data = pd.read_csv('zip_code_database.csv', dtype='object')
# the test data is your raw data, and it has 33122 records, you can use any test data you want
test_data = pd.read_csv('uszips.csv', dtype='object')


In [3]:
# sets of vilid zip codes and cities from the tranning data
valid_zip = set(train_data['zip'].tolist())
valid_city = set(train_data['primary_city'].tolist())
valid_zip = set([x.zfill(5) for x in valid_zip])

In [4]:
def predict_zip_column_index(test_data):
    # matching_num is the list to store how many records are vilid zip codes in each column
    matching_nums = []

    # iterate each column
    for column in test_data:
        count = 0
        # iterate each value in th ecolumn
        for i, row_value in test_data[column].iteritems():
            if re.fullmatch('\d\d\d\d\d', str(row_value)) or re.fullmatch('\d\d\d\d\d-\d\d\d\d', str(row_value)):
            # if len(str(row_value)) == 5 and str(row_value).isdigit():
                # if zipcodes.is_real(str(row_value)):
                if row_value[:5] in valid_zip:
                    # if it is a valid zip code, count ++
                    count += 1
        # append total valid zipcode count into the list
        matching_nums.append(count)
    print(matching_nums)
    return 'predicted index is ' + str(matching_nums.index(max(matching_nums)))

In [5]:
predict_zip_column_index(test_data)
# matching_nums
# zip, lat, lng, city, state_id, state_name, zcta, parent_zcta, population, density, county_fips, county_name, county_weights, county_names_all, county_fips_all, imprecise, military, timezone
# [33121, 0, 0, 0, 0, 0, 0, 0, 4342, 14, 20243, 0, 0, 0, 14923, 0, 0, 0]
# it is a good test dataset because you can see the noises like population and county_fips are similar to a zipcode. 
# we can see index 0, which is the zip code, has the highest matching count, so it is correctely classfied as zip code

[33121, 0, 0, 0, 0, 0, 0, 0, 4342, 14, 20243, 0, 0, 0, 14923, 0, 0, 0]


'predicted index is 0'

# 2. Missing data in zip code

In [6]:
test_data_copy = test_data.copy()

In [7]:
# 10% missing rate
rate_persent = 10
row_num = len(test_data_copy) * rate_persent // 100
selected_rows = random.sample(range(len(test_data_copy)), row_num)
len(selected_rows) # 1656 missing values
for row in selected_rows:
     test_data_copy['zip'][row] = ''

In [8]:
predict_zip_column_index(test_data_copy)

[29809, 0, 0, 0, 0, 0, 0, 0, 4342, 14, 20243, 0, 0, 0, 14923, 0, 0, 0]


'predicted index is 0'

In [9]:
# missing value does not affect the perfermance as long as the count of the remaining rows is highers than other column 

### Note: the above should be sufficient enough for zip code classfication.
### However, bind with other columns like cities can increase the confidence.
### Nevertheless, it doesn't hurt if city column is not available.

# 2. City Recognizer

In [10]:
def predict_city_column_index(test_data):
    # matching_num is the list to store how many records are vilid city names in each column
    matching_nums = []

    # iterate each column
    for column in test_data:
        count = 0
        # iterate each value in th ecolumn
        for i, row_value in test_data[column].iteritems():
            if row_value in valid_city:
                # if it is a city name, count ++
                count += 1
        # append total city name count into the list
        matching_nums.append(count)
    print(matching_nums)
    return 'predicted index is ' + str(matching_nums.index(max(matching_nums)))

In [11]:
predict_city_column_index(test_data)

[0, 0, 0, 32947, 0, 13998, 0, 0, 0, 0, 0, 26747, 0, 19369, 0, 0, 0, 0]


'predicted index is 3'

### Note: need city name to zip code data for the next step
### city names may have duplicates (not one to one maping)

# 4. use state zip prefix to increase the condifence of the rule-based model

In [12]:
zip_to_state = {}
zip_code = train_data['zip']
state = train_data['state']

In [13]:
# zipcode tp state mapping
for z, s in zip(zip_code, state):
    zip_to_state[z] = s

In [14]:
state_prefix = {}

In [15]:
# state to zip prefix mapping
for z, s in zip_to_state.items():
    if s not in state_prefix:
        state_prefix[s] = [z[:3]]
    else:
        if z[:3] not in state_prefix[s]:
            state_prefix[s].append(z[:3])

In [23]:
classfied_state_column = 'state_id'

def predict_zip_column_index_enhanced(test_data, classfied_state_column):
    # matching_num is the list to store how many records are vilid zip codes in each column
    matching_nums = []

    # iterate each column
    for column in test_data:
        count = 0
        # iterate each value in th ecolumn
        for i, row_value in test_data[column].iteritems():
            if re.fullmatch('\d\d\d\d\d', str(row_value)) or re.fullmatch('\d\d\d\d\d-\d\d\d\d', str(row_value)):
                if row_value[:5] in valid_zip:
                    # check prefix
                    if test_data[classfied_state_column][i] in state_prefix:
                        if row_value[:3] in state_prefix[test_data[classfied_state_column][i]]:
                            count += 1
        # append total valid zipcode count into the list
        matching_nums.append(count)
    print(matching_nums)
    return 'predicted index is ' + str(matching_nums.index(max(matching_nums)))

# time complexity O(mn), where m is the length of the records, n is the length of the columns
predict_zip_column_index_enhanced(test_data, classfied_state_column)

[33121, 0, 0, 0, 0, 0, 0, 0, 163, 2, 0, 0, 0, 0, 0, 0, 0, 0]


'predicted index is 0'

### previous count: [33121, 0, 0, 0, 0, 0, 0, 0, 4342, 14, 20243, 0, 0, 0, 14923, 0, 0, 0]
### current count using zip prefix: [33121, 0, 0, 0, 0, 0, 0, 0, 163, 2, 0, 0, 0, 0, 0, 0, 0, 0]
### this enhanced edition is competent to filter noices(values that are similar to zip code) in the dataset

# 5. Generate fake test data set

In [24]:
from faker import Faker
fake = Faker()

In [25]:
# prefix to state mapping
prefix_to_state = {}
for k, v in state_prefix.items():
    for i in v:
        prefix_to_state[i] = k

In [26]:
# generate random zip code and state
random_zip_code = [fake.postcode() for x in range (10000)]
random_state = []
for i in range(len(random_zip_code)):
    if random_zip_code[i][:3] in prefix_to_state:
        random_state.append(prefix_to_state[random_zip_code[i][:3]])
    else:
        random_state.append('')

In [27]:
# generate the fake dataset
fake_data = [ {'record_id': 'dba ' + str(x), 'first_name': fake.first_name(), 'last_name': fake.last_name(), 'ssn': fake.ssn(), 'zip':random_zip_code[x], 'phone': fake.phone_number(), 'dob': fake.date_of_birth(minimum_age=18, maximum_age=60), 'address': fake.street_address(), 'city': fake.city(), 'state': random_state[x], 'random_int': str(random.randint(10000, 99999))} for x in range(10000)]

In [28]:
fake_data = pd.DataFrame(fake_data)
fake_data.head(2)
# state column has missing values

,record_id,first_name,last_name,ssn,zip,phone,dob,address,city,state,random_int
0,dba 0,Alyssa,Hall,813-69-0833,88673,307-210-7529,1963-12-15,578 Garcia Hollow Apt. 539,Dillonburgh,,39195
1,dba 1,Kristy,Winters,874-52-6121,06616,+1-259-547-6406x6683,1960-10-10,42713 Wilson Path,West Cindyton,CT,70026


In [29]:
predict_zip_column_index_enhanced(fake_data, 'state')

[0, 0, 0, 0, 4254, 0, 0, 0, 0, 0, 105]


'predicted index is 4'

### the result is greate